In [1]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader
import glob
import os
import pandas as pd
import getpass
import chromadb
from langchain_openai import OpenAIEmbeddings

jangan pake full knowledges pake 1 file aja biar hemat" 😊

In [137]:
# def load_knowledges(path='knowledges'):
def load_knowledges(path='trial2'):
    docs = list()

    txt_files = glob.glob(f'{path}/*.txt')
    for txt in txt_files:
        loader = TextLoader(txt, autodetect_encoding=True)
        docs.extend(loader.load())
    pdf_files = glob.glob(f'{path}/*.pdf')
    for pdf in pdf_files:
        loader = PyPDFLoader(pdf)
        docs.extend(loader.load())

    return docs
docs = load_knowledges()

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [13]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
import uuid

# Simple

In [12]:
def simple_retriever_setup(docs, add_documents=True, delete_existed_collection=False):
    collection_name = "simple"
    # splitting
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1500, chunk_overlap=500)
    chunks = text_splitter.split_documents(docs)

    # setup chromadb
    client = chromadb.PersistentClient()
    if delete_existed_collection:
        client.delete_collection(collection_name)
    collection = client.get_or_create_collection(name=collection_name, metadata={"hnsw:space": "cosine"})

    # embeddings model
    embedding_model = OpenAIEmbeddings()

    # add chunks into collection
    if add_documents:
        ids = [str(uuid.uuid1()) for _ in range(len(chunks))]
        documents = [c.page_content for c in chunks]
        embeddings = embedding_model.embed_documents(documents)
        metadatas = [c.metadata for c in chunks]
        collection.add(
            ids=ids,
            documents=documents,
            embeddings=embeddings,
            metadatas=metadatas
        )

    docsearch = Chroma(persist_directory='chroma', collection_name=collection_name, embedding_function=embedding_model)
    retriever = docsearch.as_retriever(search_kwargs={"k":3}, search_type="similarity")

    return retriever

In [13]:
retriever = simple_retriever_setup(docs)

In [16]:
retriever.invoke('teknologi sains data')

[Document(page_content='b. Menyempurnakan penelitian yang berjalan terus-menerus;\nc. Bersifat\tjujur,\tprofesional, berperikemanusiaan dan memperhatikan faktor-faktor ketepatan, seksama dan cermat, religious, serta tidak mengandung ketimpangan gender;\nd. Memberikan penemuan yang baru;\ne. Bermanfaat\tbagi\tUNAIR\tsecara\tilmiah, institusional, dan finansial.\nf. Berbasis kompetensi logis; dan\ng. Mengingat aspek akuntabilitas.\n\n2. Hubungan Peneliti dengan Mahasiswa\n\nDalam melakukan penelitian, seorang dosen wajib melibatkan mahasiswa sebagai bentuk pemenuhan syarat akademik atau arena pembelajaran, aktualitas kompetensi bidang keilmuan, dan pengembangan pribadi.\n3. Penelitian Dasar dan Terapan\n\nSebagai peneliti, seorang dosen wajib:\na. Mengarahkan penelitian untuk kemajuan ilmu pengetahuan dan/atau perolehan hak paten untuk mendorong perkembangan industri nasional;\nb. Meningkatkan ketahanan nasional melalui penggalian sumber daya alam; dan\nc. Menyinergikan berbagai macam di

# Semantic

In [19]:
from langchain_experimental.text_splitter import SemanticChunker

In [38]:
# Semantic
def semantic_splitting_retriever_setup(docs, add_documents=True, delete_existed_collection=False):
    collection_name="semantic_splitting"

    # splitting
    text_splitter = SemanticChunker(OpenAIEmbeddings())
    chunks = text_splitter.split_documents(docs)

    # setup chromadb
    client = chromadb.PersistentClient()
    if delete_existed_collection:
        client.delete_collection(collection_name)
    collection = client.get_or_create_collection(name=collection_name, metadata={"hnsw:space": "cosine"})

    # embeddings model
    embedding_model = OpenAIEmbeddings()

    # add chunks into collection
    if add_documents:
        ids = [str(uuid.uuid1()) for _ in range(len(chunks))]
        documents = [c.page_content for c in chunks]
        embeddings = embedding_model.embed_documents(documents)
        metadatas = [c.metadata for c in chunks]    
        collection.add(
            ids=ids,
            documents=documents,
            embeddings=embeddings,
            metadatas=metadatas
        )

    docsearch = Chroma(persist_directory='chroma', collection_name=collection_name, embedding_function=embedding_model)
    retriever = docsearch.as_retriever(search_kwargs={"k":3}, search_type="similarity")

    return retriever    

In [40]:
semantic_splitting_retriever = semantic_splitting_retriever_setup(
    docs=docs, 
    delete_existed_collection=True
)

In [41]:
semantic_splitting_retriever.invoke('dosen')

[Document(page_content='Menghargai penemuan dan pendapat akademisi lain; dan\nc. Tidak semata-mata untuk kepentingan pribadi. Kedua, seorang dosen wajib menjunjung kebebasan mimbar akademik, yaitu kebebasan menyampaikan pikiran dan pendapat dalam lingkungan serta forum akademik dalam bentuk ceramah, seminar, dan kegiatan ilmiah lainnya sesuai dengan norma serta kaidah keilmuan. Seorang dosen juga memiliki kewajiban untuk selalu mawas diri dan mengevaluasi kinerja sebagai dosen dalam upayanya membina dan mengembangkan karier akademik dan profesi. Ketiga, seorang dosen wajib menumbuhkembangkan suasana akademik di lingkungan kerjanya. Dalam berkomunikasi, baik secara lisan maupun tertulis, dosen menggunakan bahasa yang sopan dan santun, tidak emosional, berpikir jernih, serta tidak menyinggung perasaan orang lain. Keempat, seorang dosen wajib memelihara dan menumbuhkembangkan masyarakat akademik antardosen, yaitu:\na. Memegang teguh dan menghormati hak dan kebebasan akademik serta hak keb

# RAPTOR

In [44]:
groq_api = os.getenv("GROQ_API_KEY")
openai_api = os.getenv("OPENAI_API_KEY")

## function raptor

In [60]:
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import umap
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture
import time

RANDOM_SEED = 224  # Fixed seed for reproducibility

### --- Code from citations referenced above (added comments and docstrings) --- ###


def global_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    """
    Perform global dimensionality reduction on the embeddings using UMAP.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - n_neighbors: Optional; the number of neighbors to consider for each point.
                   If not provided, it defaults to the square root of the number of embeddings.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


def local_cluster_embeddings(
    embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
) -> np.ndarray:
    """
    Perform local dimensionality reduction on the embeddings using UMAP, typically after global clustering.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - num_neighbors: The number of neighbors to consider for each point.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    return umap.UMAP(
        n_neighbors=num_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


def get_optimal_clusters(
    embeddings: np.ndarray, max_clusters: int = 50, random_state: int = RANDOM_SEED
) -> int:
    """
    Determine the optimal number of clusters using the Bayesian Information Criterion (BIC) with a Gaussian Mixture Model.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - max_clusters: The maximum number of clusters to consider.
    - random_state: Seed for reproducibility.

    Returns:
    - An integer representing the optimal number of clusters found.
    """
    max_clusters = min(max_clusters, len(embeddings))
    n_clusters = np.arange(1, max_clusters)
    bics = []
    for n in n_clusters:
        gm = GaussianMixture(n_components=n, random_state=random_state)
        gm.fit(embeddings)
        bics.append(gm.bic(embeddings))
    return n_clusters[np.argmin(bics)]


def GMM_cluster(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """
    Cluster embeddings using a Gaussian Mixture Model (GMM) based on a probability threshold.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - threshold: The probability threshold for assigning an embedding to a cluster.
    - random_state: Seed for reproducibility.

    Returns:
    - A tuple containing the cluster labels and the number of clusters determined.
    """
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state)
    gm.fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters


def perform_clustering(
    embeddings: np.ndarray,
    dim: int,
    threshold: float,
) -> List[np.ndarray]:
    """
    Perform clustering on the embeddings by first reducing their dimensionality globally, then clustering
    using a Gaussian Mixture Model, and finally performing local clustering within each global cluster.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for UMAP reduction.
    - threshold: The probability threshold for assigning an embedding to a cluster in GMM.

    Returns:
    - A list of numpy arrays, where each array contains the cluster IDs for each embedding.
    """
    if len(embeddings) <= dim + 1:
        # Avoid clustering when there's insufficient data
        return [np.array([0]) for _ in range(len(embeddings))]

    # Global dimensionality reduction
    reduced_embeddings_global = global_cluster_embeddings(embeddings, dim)
    # Global clustering
    global_clusters, n_global_clusters = GMM_cluster(
        reduced_embeddings_global, threshold
    )

    all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
    total_clusters = 0

    # Iterate through each global cluster to perform local clustering
    for i in range(n_global_clusters):
        # Extract embeddings belonging to the current global cluster
        global_cluster_embeddings_ = embeddings[
            np.array([i in gc for gc in global_clusters])
        ]

        if len(global_cluster_embeddings_) == 0:
            continue
        if len(global_cluster_embeddings_) <= dim + 1:
            # Handle small clusters with direct assignment
            local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
            n_local_clusters = 1
        else:
            # Local dimensionality reduction and clustering
            reduced_embeddings_local = local_cluster_embeddings(
                global_cluster_embeddings_, dim
            )
            local_clusters, n_local_clusters = GMM_cluster(
                reduced_embeddings_local, threshold
            )

        # Assign local cluster IDs, adjusting for total clusters already processed
        for j in range(n_local_clusters):
            local_cluster_embeddings_ = global_cluster_embeddings_[
                np.array([j in lc for lc in local_clusters])
            ]
            indices = np.where(
                (embeddings == local_cluster_embeddings_[:, None]).all(-1)
            )[1]
            for idx in indices:
                all_local_clusters[idx] = np.append(
                    all_local_clusters[idx], j + total_clusters
                )

        total_clusters += n_local_clusters

    return all_local_clusters


### --- Our code below --- ###


def embed(texts):
    """
    Generate embeddings for a list of text documents.

    This function assumes the existence of an `embd` object with a method `embed_documents`
    that takes a list of texts and returns their embeddings.

    Parameters:
    - texts: List[str], a list of text documents to be embedded.

    Returns:
    - numpy.ndarray: An array of embeddings for the given text documents.
    """
    text_embeddings = embd.embed_documents(texts)
    text_embeddings_np = np.array(text_embeddings)
    return text_embeddings_np


def embed_cluster_texts(texts):
    """
    Embeds a list of texts and clusters them, returning a DataFrame with texts, their embeddings, and cluster labels.

    This function combines embedding generation and clustering into a single step. It assumes the existence
    of a previously defined `perform_clustering` function that performs clustering on the embeddings.

    Parameters:
    - texts: List[str], a list of text documents to be processed.

    Returns:
    - pandas.DataFrame: A DataFrame containing the original texts, their embeddings, and the assigned cluster labels.
    """
    text_embeddings_np = embed(texts)  # Generate embeddings
    cluster_labels = perform_clustering(
        text_embeddings_np, 10, 0.1
    )  # Perform clustering on the embeddings
    df = pd.DataFrame()  # Initialize a DataFrame to store the results
    print("text", texts)
    print("embd", list(text_embeddings_np))
    print("cluster",cluster_labels)
    df["text"] = texts  # Store original texts
    df["embd"] = list(text_embeddings_np)  # Store embeddings as a list in the DataFrame
    df["cluster"] = cluster_labels  # Store cluster labels
    print(len(docs_metadata))
    print(docs_metadata)
    print(len(texts))
    if len(docs_metadata) == len(texts):
        df["metadata"] = docs_metadata
    else:
        df["metadata"] = None
    return df


def fmt_txt(df: pd.DataFrame) -> str:
    """
    Formats the text documents in a DataFrame into a single string.

    Parameters:
    - df: DataFrame containing the 'text' column with text documents to format.

    Returns:
    - A single string where all text documents are joined by a specific delimiter.
    """
    unique_txt = df["text"].tolist()
    return "--- --- \n --- --- ".join(unique_txt)


def embed_cluster_summarize_texts(
    texts: List[str], level: int
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Embeds, clusters, and summarizes a list of texts. This function first generates embeddings for the texts,
    clusters them based on similarity, expands the cluster assignments for easier processing, and then summarizes
    the content within each cluster.

    Parameters:
    - texts: A list of text documents to be processed.
    - level: An integer parameter that could define the depth or detail of processing.

    Returns:
    - Tuple containing two DataFrames:
      1. The first DataFrame (`df_clusters`) includes the original texts, their embeddings, and cluster assignments.
      2. The second DataFrame (`df_summary`) contains summaries for each cluster, the specified level of detail,
         and the cluster identifiers.
    """

    # Embed and cluster the texts, resulting in a DataFrame with 'text', 'embd', and 'cluster' columns
    df_clusters = embed_cluster_texts(texts)

    # Prepare to expand the DataFrame for easier manipulation of clusters
    expanded_list = []

    # Expand DataFrame entries to document-cluster pairings for straightforward processing
    for index, row in df_clusters.iterrows():
        for cluster in row["cluster"]:
            expanded_list.append(
                {"text": row["text"], "embd": row["embd"], "cluster": cluster}
            )

    # Create a new DataFrame from the expanded list
    expanded_df = pd.DataFrame(expanded_list)

    # Retrieve unique cluster identifiers for processing
    all_clusters = expanded_df["cluster"].unique()

    print(f"--Generated {len(all_clusters)} clusters--")

    # Summarization
    template = """Ini adalah kumpulan dokumen Fakultas Teknologi Maju dan Multidisplin (FTMM), Universitas Airlangga
    
    Buatlah ringkasan secara detail dari dokumen tersebut dengan menggunakan Bahasa Indonesia. 
    
    Documentation:
    {context}
    """
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | StrOutputParser()

    # Format text within each cluster for summarization
    summaries = []
    for i in all_clusters:
        df_cluster = expanded_df[expanded_df["cluster"] == i]
        formatted_txt = fmt_txt(df_cluster)
        summaries.append(chain.invoke({"context": formatted_txt}))
        time.sleep(60)

    # Create a DataFrame to store summaries with their corresponding cluster and level
    df_summary = pd.DataFrame(
        {
            "summaries": summaries,
            "level": [level] * len(summaries),
            "cluster": list(all_clusters),
        }
    )

    return df_clusters, df_summary


def recursive_embed_cluster_summarize(
    texts: List[str], level: int = 1, n_levels: int = 3
) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    Recursively embeds, clusters, and summarizes texts up to a specified level or until
    the number of unique clusters becomes 1, storing the results at each level.

    Parameters:
    - texts: List[str], texts to be processed.
    - level: int, current recursion level (starts at 1).
    - n_levels: int, maximum depth of recursion.

    Returns:
    - Dict[int, Tuple[pd.DataFrame, pd.DataFrame]], a dictionary where keys are the recursion
      levels and values are tuples containing the clusters DataFrame and summaries DataFrame at that level.
    """
    results = {}  # Dictionary to store results at each level

    # Perform embedding, clustering, and summarization for the current level
    df_clusters, df_summary = embed_cluster_summarize_texts(texts, level)

    # Store the results of the current level
    results[level] = (df_clusters, df_summary)

    # Determine if further recursion is possible and meaningful
    unique_clusters = df_summary["cluster"].nunique()
    if level < n_levels and unique_clusters > 1:
        # Use summaries as the input texts for the next level of recursion
        new_texts = df_summary["summaries"].tolist()
        next_level_results = recursive_embed_cluster_summarize(
            new_texts, level + 1, n_levels
        )

        # Merge the results from the next level into the current results dictionary
        results.update(next_level_results)

    return results

## final code

In [169]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_groq import ChatGroq

docs_metadata = [c.metadata for c in docs] 
docs_texts = [d.page_content for d in docs]

embd = OpenAIEmbeddings(model = "text-embedding-3-small")
model = ChatGroq(temperature=0, groq_api_key=groq_api, model_name="llama3-70b-8192")

leaf_texts = docs_texts
results = recursive_embed_cluster_summarize(leaf_texts, level=1, n_levels=3)

leaf = results[1][0]
leaf['cluster'] = leaf['cluster'].apply(lambda x: int(x[0]))
leaf['metadata'] = leaf.apply(lambda row: {**row['metadata'], 'level': 2, 'cluster': row['cluster']}, axis=1)
leaf = leaf.drop(columns=['cluster'])


node_1 = results[2][0][['text','embd']]
node_2 = results[1][1]
node_2.rename(columns={'summaries': 'text'}, inplace=True)
node = pd.concat([node_1.set_index('text'), node_2.set_index('text')], axis=1, join='inner').reset_index()
node['metadata'] = node.apply(lambda row: {'level': int(row['level']), 'cluster': int(row['cluster'])}, axis=1)
node = node.drop(columns=['level','cluster'])

df = pd.concat([leaf, node], ignore_index=True)

text ['Alur Pendaftaran Online Mahasiswa Baru\nUniversitas Airlangga\n_____________________________________________________________________________________\nBerlaku untuk semua pendaftaran semua strata (Penerimaan Dalam negeri)\nA. Registrasi Akun Pendaftaran\nSebelum melakukan pengisian formulir pendaftaran online, Anda diwajibkan mempunyai akun\npendaftaran. Akun ini akan digunakan sebagai login untuk masuk ke dalam Sistem Pendaftaran\nOnline Universitas Airlangga.\nLangkah-langkahnya adalah sebagai berikut :\n1. Buka web pendaftaran Universitas Airlangga : http://pendaftaran.unair.ac.id.\n2. Klik menu Registrasi.\n3. Setelah tampilan muncul, kemudian isikan Nama, No Telp / HP (disarankan HP), Email,\nPassword yang diinginkan. Email & Password ini akan digunakan untuk login\n4. Klik tombol [Registrasi], Jika sudah melakukan regitrasi peserta dapat langsung\nmelakukan login untuk pendaftaran.\n5. Proses registrasi akun pendaftaran sudah selesai, anda hanya cukup melakukan proses ini\n

In [173]:
df

,text,embd,metadata
0,Alur Pendaftaran Online Mahasiswa Baru\nUniver...,"[-0.01081357286995246, 0.005985540989438113, -...",{'source': 'trial2\Alur Pendaftaran Online Mah...
1,Alur Pengajuan Legalisir dan Transkrip Nilai\n...,"[-0.014581798877069525, 0.002589663544153892, ...",{'source': 'trial2\Alur Pengajuan Legalisir at...
2,Alur Pengajuan Surat Keterangan Mahasiswa Akti...,"[-0.02177123244970739, 0.020850217799000127, -...","{'source': 'trial2\Alur Pengajuan SKMA.txt', '..."
3,Alur Pengajuan Surat Keterangan\n1. Mengunduh ...,"[-0.0260616624636963, -0.02303489214509142, 0....",{'source': 'trial2\Alur Pengajuan Surat Ketera...
4,Alur Pengajuan Surat Rekomendasi\n1. Mengunduh...,"[-0.02985882960930659, -0.032385192980383594, ...",{'source': 'trial2\Alur Pengajuan Surat Rekome...
5,Profil Program Studi\nRekayasa Nanoteknologi \...,"[-0.009220830233566544, 0.016896706200396287, ...",{'source': 'trial2\rekayasa-nanoteknologi-prog...
6,Staf Dosen Program StudiTeknik Robotika dan Ke...,"[0.023594434367419276, 0.017690409668919296, -...",{'source': 'trial2\staf dosen prodi teknik rob...
7,Staf Dosen Prodi Rekayasa Nanoteknologi \n\nKo...,"[0.006666185918649993, -0.013416420907187455, ...",{'source': 'trial2\staf-dosen-prodi-rekayasa-n...
8,\nStaf Dosen Program StudiTeknik Industri \nKo...,"[-0.02285588941319379, -0.028882671603827636, ...",{'source': 'trial2\staf-dosen-prodi-teknik-ind...
9,\nStaf Dosen Program StudiTeknologi Sains Dat...,"[-0.007149562944930877, -0.0007635139385991699...",{'source': 'trial2\staf-dosen-prodi-teknologi-...


In [171]:
collection_name="raptor3"
client = chromadb.PersistentClient()
# client.delete_collection(collection_name)
collection = client.get_or_create_collection(name=collection_name, metadata={"hnsw:space": "cosine"})

ids = [str(uuid.uuid1()) for _ in range(len(df))]
documents = df['text'].to_list()
df['embd'] = df['embd'].apply(lambda x: x.tolist())
embeddings = df['embd'].to_list()
metadatas = df['metadata'].to_list()
collection.add(
    ids=ids,
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas,
)

In [176]:
docsearch = Chroma(persist_directory='chroma', collection_name=collection_name, embedding_function=embd)

# Level 1
retriever_lv1 = docsearch.as_retriever(search_kwargs={"k":1, "filter":{"level":1}}, search_type="similarity")
result_lv1 = retriever_lv1.invoke('tenaga kependidikan')
cluster = result_lv1[0].metadata['cluster']
print(cluster)

# Level 2
retriever_lv2 = docsearch.as_retriever(search_kwargs={"k":3,"filter":{'$and': [{'level': 2}, {'cluster': cluster}]}}, search_type="similarity")
result = retriever_lv2.invoke('tenaga kependidikan')
result

0


[Document(page_content='Tenaga Kependidikan \nProfil Tenaga KependidikanFakultas Teknologi Maju dan Multidisiplin \nKepala Bagian Tata Usaha \nDwi Sulistyarini, S.H \nKepala Subbagian Sarana dan Prasarana \nBoedi Rahardjo, S.Sos \nVinanci Intan Widriani, S.M \nDian Kriswanto \nMirna Putri Anggraeni, S.Si \nYunus Rizki Setiawan, S.EI. \nAndri Hariyanto, S.Sos \nYuliana Ariandini Ayuningtiyas, S.IIP \nYulia Noor Azizah, S.Sosio. \nMohammad Ichsan Herdito \nKepala Subbagian SDM dan Keuangan \nWilujeng Noorfianti,S.E \nWikca Pratita, S.M., M.SM \nErzalina Ayu Satya Megananda, S.Si \nR. Muhammad Akbar Bagus Prakoso, Amd.Bns \nKepala Subbagian Akademik dan Kemahasiswaan \nNurina Oktiviani, S.E \nAndy S. Nugroho, S.Kom \nAhmad Dwi Cahya Oktavian, S.T \nZuhairrizqi Elfad, S.Tr.T \nNovianti Nurfauzjiah, S.Si. \nDevi Ruchmana Rachmawati, S.Si., M.Si. \nDhini Dwi Kartikawati, A.Md. Lib. \nM. Farras Taufiq Atsany, S.T \nVivin Safitri, S.T \nEliesa Chusnul Aulia, S.M \nFivien Agustina, S.Si. \nWind

In [195]:
# RAPTOR
def raptor_retriever_setup(docs, add_documents=False, delete_existed_collection=False):
    collection_name="raptor"
    # collection_name="raptor3"

    # setup chromadb
    client = chromadb.PersistentClient()
    if delete_existed_collection:
        client.delete_collection(collection_name)
    collection = client.get_or_create_collection(name=collection_name, metadata={"hnsw:space": "cosine"})

    # embeddings model
    embd = OpenAIEmbeddings(model = "text-embedding-3-small")
    model = ChatGroq(temperature=0, groq_api_key=groq_api, model_name="llama3-70b-8192")

    if add_documents:
        docs_metadata = [c.metadata for c in docs] 
        docs_texts = [d.page_content for d in docs]

        results = recursive_embed_cluster_summarize(docs_texts, level=1, n_levels=3)

        leaf = results[1][0]
        leaf['cluster'] = leaf['cluster'].apply(lambda x: int(x[0]))
        leaf['metadata'] = leaf.apply(lambda row: {**row['metadata'], 'level': 2, 'cluster': row['cluster']}, axis=1)
        leaf = leaf.drop(columns=['cluster'])

        node_1 = results[2][0][['text','embd']]
        node_2 = results[1][1]
        node_2.rename(columns={'summaries': 'text'}, inplace=True)
        node = pd.concat([node_1.set_index('text'), node_2.set_index('text')], axis=1, join='inner').reset_index()
        node['metadata'] = node.apply(lambda row: {'level': int(row['level']), 'cluster': int(row['cluster'])}, axis=1)
        node = node.drop(columns=['level','cluster'])

        df = pd.concat([leaf, node], ignore_index=True)

        ids = [str(uuid.uuid1()) for _ in range(len(df))]
        documents = df['text'].to_list()
        df['embd'] = df['embd'].apply(lambda x: x.tolist())
        embeddings = df['embd'].to_list()
        metadatas = df['metadata'].to_list()
        collection.add(
            ids=ids,
            documents=documents,
            embeddings=embeddings,
            metadatas=metadatas,
        )

    docsearch = Chroma(persist_directory='chroma', collection_name=collection_name, embedding_function=embd)

    return docsearch

def raptor_retriever(docsearch, question, k = 2):
    #Level 1
    retriever_lv1 = docsearch.as_retriever(search_kwargs={"k":1, "filter":{"level":1}}, search_type="similarity")
    result_lv1 = retriever_lv1.invoke(question)
    cluster = result_lv1[0].metadata['cluster']
    print(cluster)

    # Level 2
    retriever_lv2 = docsearch.as_retriever(search_kwargs={"k":k,"filter":{'$and': [{'level': 2}, {'cluster': cluster}]}}, search_type="similarity")
    result = retriever_lv2.invoke(question)
    return result 

In [193]:
docsearch = raptor_retriever_setup(docs,add_documents=True, delete_existed_collection=False) 
# raptor_retriever(docsearch, question = "staf dosen teknologi sains data", k = 3)

text ['Alur Pendaftaran Online Mahasiswa Baru\nUniversitas Airlangga\n_____________________________________________________________________________________\nBerlaku untuk semua pendaftaran semua strata (Penerimaan Dalam negeri)\nA. Registrasi Akun Pendaftaran\nSebelum melakukan pengisian formulir pendaftaran online, Anda diwajibkan mempunyai akun\npendaftaran. Akun ini akan digunakan sebagai login untuk masuk ke dalam Sistem Pendaftaran\nOnline Universitas Airlangga.\nLangkah-langkahnya adalah sebagai berikut :\n1. Buka web pendaftaran Universitas Airlangga : http://pendaftaran.unair.ac.id.\n2. Klik menu Registrasi.\n3. Setelah tampilan muncul, kemudian isikan Nama, No Telp / HP (disarankan HP), Email,\nPassword yang diinginkan. Email & Password ini akan digunakan untuk login\n4. Klik tombol [Registrasi], Jika sudah melakukan regitrasi peserta dapat langsung\nmelakukan login untuk pendaftaran.\n5. Proses registrasi akun pendaftaran sudah selesai, anda hanya cukup melakukan proses ini\n

text ['Berikut adalah ringkasan dokumen Fakultas Teknologi Maju dan Multidisiplin (FTMM), Universitas Airlangga:\n\n**Alur Pendaftaran Online Mahasiswa Baru**\n\n1. Registrasi Akun Pendaftaran: Calon mahasiswa harus membuat akun pendaftaran online dengan mengisi formulir registrasi di website Universitas Airlangga.\n2. Pengisian Formulir Pendaftaran dan Upload Berkas Persyaratan: Setelah registrasi, calon mahasiswa harus mengisi formulir pendaftaran online dan mengupload berkas persyaratan yang dibutuhkan.\n3. Pembayaran Voucher dan Cetak Kartu Ujian: Setelah proses verifikasi selesai, calon mahasiswa harus melakukan pembayaran voucher dan mencetak kartu ujian.\n\n**Alur Pengajuan Legalisir dan Transkrip Nilai**\n\n1. Mengunduh template Surat Rekomendasi dan membuat Surat Permohonan.\n2. Mengirimkan berkas ke email kemahasiswaan@ftmm.unair.ac.id dengan subject "Transkrip/Legalisir KHS_Nama_NIM".\n3. Permohonan akan diproses dalam 3-7 hari kerja.\n4. Surat dalam bentuk hardcopy dapat di

In [194]:
raptor_retriever(docsearch, question = "staf dosen teknologi sains data", k = 3)

IndexError: list index out of range

In [230]:
docsearch = raptor_retriever_setup(docs,add_documents=False, delete_existed_collection=False) 
docs = raptor_retriever(docsearch, question = "dosen teknologi sains data", k = 3)
docs

2


[Document(page_content='Profil Program Studi\nTeknologi Sains Data\nVisi\nPada tahun 2030 menjadi sebuah Program Studi S1 Teknologi Sains Data yang unggul di Indonesia dalam penerapan sains data pada bidang kesehatan, sosial dan komunikasi, industri, serta bisnis dan finansial untuk mendukung perkembangan masyarakat ilmiah yang cerdas/unggul dan berbudi pekerti luhur/bermoral karimah.\n\n\n\nMisi\nMenyelenggarakan pendidikan di bidang Teknologi Sains Data yang berstandar nasional dan internasional agar mahasiswa mampu untuk berpikir kritis, berwawasan global, dan mengembangkan keterampilan dalam menjawab tantangan global\nMenyelenggarakan penelitian dan pengembangan ilmu di bidang Teknologi Sains Data yang berstandar nasional dan internasional di bidang kesehatan, sosial dan komunikasi, industri, serta bisnis dan finansial\nMenyelenggarakan pengabdian kepada masyarakat yang menunjang penerapan dan pengembangan ilmu Teknologi Sains Data yang dapat mendukung perkembangan masyarakat denga

In [228]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

# Prompt
prompt = hub.pull("rlm/rag-prompt")


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": itemgetter("context"), "question": itemgetter("question")}
    | prompt
    | model
    | StrOutputParser()
)

docsearch = raptor_retriever_setup(docs,add_documents=False, delete_existed_collection=False) 
docs = raptor_retriever(docsearch, question = question, k = 3)
context = format_docs(docs)
# print(context)

# Question
question = "jelaskan tentang Staf Dosen Program StudiTeknologi Sains Data "
rag_chain.invoke({"context": context, "question": question})

0


'The question is asking about the staff of the Data Science Program Study. The staff consists of Dr. Dwi Rantini as the coordinator, and other lecturers including Muhammad Noor Fakhruzzaman, Ratih Ardiati Ningrum, and several others.'